# LLM 성능 극대화를 위한 합성 데이터 생성 실습

## 실습 목표
데이터 부족 문제를 해결하기 위해 고품질의 합성 데이터를 생성하는는 두 가지 방법을 직접 실습합니다:

1. **명령어 미세 조정(SFT)** 데이터 생성
   - 지시사항과 응답 쌍을 자동으로 생성
   - 모델이 사용자 지시를 더 잘 따르도록 훈련

2. **직접 선호 최적화(DPO)** 데이터 생성
   - 좋은 답변과 나쁜 답변 쌍을 자동으로 생성
   - 모델의 선호도를 인간 가치에 맞게 정렬

### 사용 모델: Kakao Kanana 1.5 2.1B Instruct
- **개발사**: Kakao
- **모델 크기**: 2.1B 파라미터
- **모델 타입**: Instruct (지시사항 특화)
- **언어 지원**: 한국어 특화
- **버전**: 2505 (2025년 5월 버전)

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 ID - Kakao Kanana 1.5 2.1B
model_id = "kakaocorp/kanana-1.5-2.1b-instruct-2505"  # 또는 다른 가능한 ID

# 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else "auto",
    device_map="auto" if torch.cuda.is_available() else None,
    trust_remote_code=True,
)

print("Kanana 모델 로딩 성공!")  # 약 7분 소요

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Kanana 모델 로딩 성공!


## Instruction 데이터 자동 생성

### 실습 방법
특정 "시드(seed) 컨텍스트"를 바탕으로 LLM에게 질문(instruction)과 답변(output)을 생성하도록 하여 SFT 데이터를 자동으로 구축합니니다.

### SEED 컨텍스트 정의
데이터 생성을 위한 기반 지식이 될 텍스트입니다.   
이 정보를 바탕으로 모델이 질문과 답변을 생성합니다.  
이번 실습에서는는 '클로드 3.5 소네트'에 대한 최신 정보를 시드 컨텍스트로 사용하겠습니다.

In [ ]:
# SFT 데이터 생성을 위한 기반 지식

seed_context = """
앤트로픽이 최신 모델 '클로드 3.5 소네트'를 출시했습니다. 이 모델은 이전 주력 모델이었던 '클로드 3 오퍼스'보다 2배 빠른 속도를 자랑하며, 더 저렴한 비용으로 제공됩니다. '클로드 3.5 소네트'는 특히 추론, 코드 생성, 시각 정보 이해 능력에서 큰 발전을 이루었습니다.

주요 특징 중 하나는 '아티팩트(Artifacts)'라는 새로운 기능입니다. 사용자가 클로드에게 코드 생성을 요청하면, 결과물이 별도의 창에 나타나 바로 실행하거나 편집할 수 있습니다. 이는 개발자들의 작업 흐름을 크게 개선할 수 있는 혁신적인 기능으로 평가받고 있습니다.

또한, 비전(Vision) 성능이 뛰어나 차트나 그래프를 텍스트로 정확하게 변환하거나, 이미지 속 텍스트를 추출하는 능력이 이전 모델을 능가합니다. 앤트로픽은 '클로드 3.5 소네트'를 시작으로, 앞으로 '3.5 하이쿠'와 '3.5 오퍼스' 모델도 연내에 출시할 계획이라고 밝혔습니다.
"""

### 데이터 생성을 위한 프롬프트 템플릿 설계

#### 프롬프트 엔지니어링의 중요성
LLM에게 원하는 출력 형식을 명확히 지정하는 것이 핵심입니다.

#### 설계 원칙
1. **명확성**: 모델이 이해하기 쉬운 지시사항
2. **구체성**: 원하는 출력 형식을 명확히 지정
3. **범용성**: 다양한 주제에 적용 가능
4. **실용성**: 실제 훈련에 바로 사용 가능

#### 프롬프트 구성 요소
- **역할 정의**: 모델의 역할과 목적 명시
- **요구사항**: 구체적인 생성 규칙
- **출력 형식**: 원하는 결과 형태 지정
- **예시**: 모델이 참고할 수 있는 템플릿

In [ ]:
# SFT 데이터 생성을 위한 프롬프트 템플릿
# LLM에게 원하는 출력 형식을 명확히 지정합니다.

sft_prompt_template = f"""
다음 정보를 바탕으로 AI 훈련용 지시 데이터를 생성해주세요:

{seed_context}

이 주제의 다양한 측면을 다루는 3-5개의 질문-답변 쌍을 생성해주세요.

요구사항:
- 질문은 사용자가 실제로 궁금해할 만한 것들
- 답변은 포괄적이고 정확해야 함
- 다양한 난이도 포함
- 기본적인 질문과 고급 질문 모두 포함

각 쌍의 형식:
질문: [사용자 질문]
답변: [상세한 응답]

생성 시작:
"""

### SFT 데이터 생성 실행

#### 생성 과정
1. **토크나이징**: 프롬프트를 모델이 이해할 수 있는 형태로 변환
2. **텍스트 생성**: 모델을 통한 자동 응답 생성
3. **결과 디코딩**: 생성된 토큰을 읽기 쉬운 텍스트로 변환

#### 생성 파라미터
- **max_new_tokens**: 생성할 최대 토큰 수 (응답 길이 제어)
- **temperature**: 창의성 vs 일관성 조절 (0.7 권장)
- **do_sample**: 확률적 샘플링 사용 여부
- **repetition_penalty**: 반복 방지 계수

In [ ]:
# 토크나이저를 사용해 모델 입력 형식으로 변환
inputs = tokenizer.encode(sft_prompt_template, return_tensors="pt")


# 모델을 통해 텍스트 생성
# max_new_tokens: 생성할 최대 토큰 수
# do_sample=True: 좀 더 창의적인 답변을 위해 샘플링 사용
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens=1500,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

# 생성된 결과 디코딩
response_text = tokenizer.decode(outputs[0][inputs.shape[-1] :], skip_special_tokens=True)

print("생성된 SFT 데이터")
print(response_text)

### TO DO
prompt를 수정해보고 결과를 출력해 비교해보세요.  

In [5]:
# 아래는 기존에 사용했던 prompt 템플릿입니다.
# 어떤 부분을 수정해야 더 나은 결과를 얻을 수 있을지 여러 실험을 통해 알아봅시다.

sft_prompt_template = f"""
다음 정보를 바탕으로 AI 훈련용 지시 데이터를 생성해주세요:

{seed_context}

이 주제의 다양한 측면을 다루는 3-5개의 질문-답변 쌍을 생성해주세요.

요구사항:
- 질문은 사용자가 실제로 궁금해할 만한 것들
- 답변은 포괄적이고 정확해야 함
- 다양한 난이도 포함
- 기본적인 질문과 고급 질문 모두 포함

각 쌍의 형식:
질문: [사용자 질문]
답변: [상세한 응답]

생성 시작:
"""

In [ ]:
inputs = tokenizer.encode(sft_prompt_template, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

response_text = tokenizer.decode(outputs[0][inputs.shape[-1] :], skip_special_tokens=True)

print("생성된 SFT 데이터")
print(response_text)

## 선호도(DPO) 데이터 자동 생성

### DPO(Direct Preference Optimization)란?
DPO는 '더 나은 응답(chosen)'과 '덜 나은 응답(rejected)' 쌍을 학습하여 모델의 답변 품질을 직접적으로 향상시키는 기법입니다.

### DPO의 핵심 개념
- **선호도 학습**: 인간이 선호하는 답변 패턴 학습
- **품질 차별화**: 좋은 답변과 나쁜 답변 구분
- **가치 정렬**: 모델의 출력을 인간 가치에 맞게 조정

### 실습 방법
Distilabel을 이용해 선호도 데이터를 만드는 실습을 해보겠습니다.

### Distilabel의 장점
- **전문 도구**: DPO 데이터 생성에 특화
- **자동화**: 수동 작업 대신 자동 생성
- **품질 보장**: 고품질 선호도 데이터 생성
- **효율성**: 빠르고 일관된 결과

### 환경 설정

In [1]:
import os

from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
print("API 키가 입력되었습니다.")

API 키가 입력되었습니다.


### Distilabel과 OpenAI API를 이용한 DPO 데이터 생성

#### 접근 방법
1. **다양한 프롬프트**: 여러 주제의 질문으로 데이터 다양성 확보
2. **이중 모델 시스템**: 
   - 생성 모델: 좋은 답변과 나쁜 답변 생성
   - 평가 모델: 답변 품질 평가 및 선호도 결정
3. **파이프라인 구성**: 자동화된 데이터 생성 및 평가 과정

#### 프롬프트 선택 기준
- **다양성**: 다양한 주제와 난이도
- **실용성**: 실제 사용자가 궁금해할 만한 질문
- **평가 가능성**: 좋고 나쁨을 명확히 구분할 수 있는 주제

#### 모델 설정 전략
- **생성 모델 1**: 상대적으로 성능이 좋은 모델 (chosen 답변 생성)
- **생성 모델 2**: 상대적으로 성능이 낮은 모델 (rejected 답변 생성)
- **평가 모델**: 답변 품질을 평가하는 심판 역할

In [7]:
import os

from distilabel.llms import OpenAILLM
from distilabel.pipeline import Pipeline
from distilabel.steps import FormatTextGenerationDPO, GroupColumns, LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration, UltraFeedback

prompts = [
    "한국의 전통 시장을 배경으로 한 따뜻한 단편소설을 200자 내외로 작성해주세요.",
    "중학생도 이해할 수 있도록 인공지능의 기본 개념을 설명해주세요.",
    "재택근무와 사무실 근무의 장단점을 비교 분석해주세요.",
    "기후변화의 주요 원인과 대응방안을 3가지씩 요약해주세요.",
    "Python으로 리스트에서 중복을 제거하는 3가지 방법을 코드와 함께 설명해주세요.",
]

/tmp/ipykernel_63/3699236350.py:2: DeprecationWarning: Importing from 'distilabel.llms' is deprecated and will be removed in a version 1.7.0. Import from 'distilabel.models' instead.
  from distilabel.llms import OpenAILLM


In [ ]:
# 답변 생성을 위한 모델 정의
# 생성 모델 중 하나는 최신 모델, 다른 하나는 이전 세대 모델로 설정하여 성능 차이를 유도
llm_generator_1 = OpenAILLM(
    model="gpt-4.1-nano",  # 상대적으로 성능이 좋은 모델
    api_key=api_key,
    generation_kwargs={"max_new_tokens": 500, "temperature": 0.3},
)
llm_generator_2 = OpenAILLM(
    model="gpt-4o-mini",  # 상대적으로 성능이 낮은 모델
    api_key=api_key,
    generation_kwargs={"max_new_tokens": 500, "temperature": 0.9},  # 높은 Temperature로 오답 생성 유도
)

# 생성된 답변을 평가할 모델 정의
llm_judge = OpenAILLM(model="gpt-4.1-mini", api_key=api_key)

### 데이터셋 생성 파이프라인 정의

#### 파이프라인 구성 요소
1. **데이터 로딩**: 초기 프롬프트 데이터 준비
2. **답변 생성**: 두 개의 모델로 chosen/rejected 답변 생성
3. **결과 그룹화**: 생성된 답변들을 하나로 통합
4. **품질 평가**: 심판 모델로 답변 품질 평가
5. **형식 변환**: 최종 DPO 학습 형식으로 변환

#### 파이프라인 흐름
프롬프트 → 답변 생성(2개 모델) → 그룹화 → 평가 → DPO 형식


#### 각 단계의 역할
- **LoadDataFromDicts**: 초기 데이터 로딩
- **TextGeneration**: LLM을 통한 답변 생성
- **GroupColumns**: 여러 결과를 하나로 통합
- **UltraFeedback**: 답변 품질 평가
- **FormatTextGenerationDPO**: 최종 DPO 형식 변환

In [ ]:
# 파이프라인 정의
with Pipeline(name="dpo-generation-pipeline") as pipeline:
    initial_data = [{"instruction": p} for p in prompts]
    load_prompts = LoadDataFromDicts(name="load_prompts", data=initial_data)

    # Chosen 답변 생성 (상세한 설명)
    generate_responses_1 = TextGeneration(
        name="generate_chosen",
        llm=llm_generator_1,
        system_prompt="You are a friendly and verbose assistant that explains things in great detail.",
    )
    # Rejected 답변 생성 (간단한 답변)
    generate_responses_2 = TextGeneration(
        name="generate_rejected",
        llm=llm_generator_2,
        system_prompt="You are an assistant who only speaks about answers and omit all the process",
    )

    # 생성된 답변들을 하나로 그룹화
    group_responses = GroupColumns(
        name="group_responses",
        columns=["generation", "model_name"],
        output_columns=["generations", "model_names"],
    )

    # 심판 LLM으로 답변 품질 평가
    evaluate_responses = UltraFeedback(
        name="evaluate_responses",
        llm=llm_judge,
        aspect="overall-rating",
    )

    # DPO 학습 형식으로 변환
    format_dpo = FormatTextGenerationDPO(name="format_dpo")

    # 파이프라인 연결
    load_prompts.connect(generate_responses_1)
    load_prompts.connect(generate_responses_2)
    generate_responses_1.connect(group_responses)
    generate_responses_2.connect(group_responses)
    group_responses.connect(evaluate_responses)
    evaluate_responses.connect(format_dpo)

# 파이프라인 실행
distiset = pipeline.run(use_cache=False)

### 데이터 전처리 및 데이터프레임 변환

#### 전처리 목적
생성된 원시 데이터를 실제 DPO 훈련에 사용할 수 있는 형태로 변환합니다.

#### 처리 과정
1. **원시 데이터 확인**: Distilabel에서 생성된 데이터 구조 파악
2. **필요 컬럼 추출**: instruction, chosen, rejected 컬럼만 선택
3. **텍스트 정제**: assistant 역할의 답변 텍스트만 추출
4. **데이터 검증**: 빈 값이나 오류 데이터 확인

#### 최종 데이터 형태
- **instruction**: 사용자 질문/지시사항
- **chosen**: 선호되는 좋은 답변
- **rejected**: 선호되지 않는 나쁜 답변

#### 데이터 품질 확인
- 답변의 완성도 검증
- chosen과 rejected의 명확한 차이 확인
- 데이터 형식의 일관성 검증

In [ ]:
distiset

In [ ]:
raw_df = distiset["default"]["train"].to_pandas()
raw_df

In [ ]:
preference_data = raw_df[["instruction", "chosen", "rejected"]]
# chosen과 rejected 컬럼에서 'assistant'의 답변 텍스트만 추출
preference_data["chosen"] = preference_data["chosen"].apply(
    lambda arr: next((item["content"] for item in arr if item.get("role") == "assistant"), None)
)
preference_data["rejected"] = preference_data["rejected"].apply(
    lambda arr: next((item["content"] for item in arr if item.get("role") == "assistant"), None)
)
preference_data

## 마무리 및 내용 정리

### 실습을 통해 배운 것들

#### 1. 합성 데이터의 필요성
- 고품질의 학습 데이터는 LLM 성능에 필수적
- 데이터가 부족할 때 합성 데이터는 훌륭한 대안
- 자동화된 데이터 생성으로 효율성 극대화

#### 2. SFT 데이터 생성
- LLM을 이용해 특정 주제에 대한 (지시, 응답) 쌍을 자동으로 생성
- 모델이 지시를 잘 따르도록 만드는 방법 학습
- 프롬프트 엔지니어링의 중요성 체험

#### 3. DPO 데이터 생성
- Distilabel을 활용한 자동화된 선호도 데이터 생성
- 모델의 선호도를 인간의 가치에 맞게 정렬하는 방법
- 파이프라인 기반의 체계적인 데이터 생성 과정

### 실무 적용 방안
이러한 기법들은 여러분이 LLM을 특정 목적에 맞게 고도화하고, 데이터 부족 문제를 해결하는 데 강력한 도구가 될 것입니다.